## Lectura e integración de datos de análisis de toxina paralizante en moluscos (TPM)

**Importación de librerias**

In [1]:
import pandas as pd
import numpy as np

**Lectura del primer dataset**

In [16]:
df_antiguo = pd.read_excel("antiguo_unido.xlsx")
df_antiguo.head()

,Fecha,Zona_Clasificada,Resultado_cat
0,2021-01-06,ARTF 005,0
1,2021-01-06,ARTF 002,0
2,2021-01-06,ARTF 006,0
3,2021-01-06,ARTF 001,0
4,2021-01-06,ARTF 003,0


**Lectura de nuevos datos**

In [17]:
df_nuevo = pd.read_excel("limpio_unido.xlsx")
df_nuevo.head()

,Fecha,Zona_Clasificada,Resultado_cat
0,2022-03-28,ARTF 001 PUNTA PARANÁ,2
1,2022-03-28,ARTF 002 BAJO BROWN,2
2,2022-03-28,ARTF 002 PUNTA LÁPIZ,2
3,2022-03-28,ARTF 002 PASO GUARANÍ,2
4,2022-03-28,ARTF 002 BAHÍA BROWN ENTRADA,2


**Unión de ambos sets de datos**

In [18]:
df_concatenado = pd.concat([df_antiguo, df_nuevo], ignore_index=True)
df_concatenado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Fecha             1000 non-null   datetime64[ns]
 1   Zona_Clasificada  1000 non-null   object        
 2   Resultado_cat     1000 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 23.6+ KB


In [19]:
df_concatenado.head()

,Fecha,Zona_Clasificada,Resultado_cat
0,2021-01-06,ARTF 005,0
1,2021-01-06,ARTF 002,0
2,2021-01-06,ARTF 006,0
3,2021-01-06,ARTF 001,0
4,2021-01-06,ARTF 003,0


**Ordenamiento del set de datos y re-indexación para numerar las filas nuevamente.**

In [20]:
df_ordenado = df_concatenado.sort_values('Fecha').reset_index(drop=True)
df_ordenado.head()

,Fecha,Zona_Clasificada,Resultado_cat
0,2021-01-06,ARTF 005,0
1,2021-01-06,ARTF 002,0
2,2021-01-06,ARTF 006,0
3,2021-01-06,ARTF 001,0
4,2021-01-06,ARTF 003,0


**Agrupacion por fecha tomando el valor maximo de categoria (0: no detectable, 1: apto p/consumo, 2: no apto para consumo)**

In [21]:
df_resumen = df_ordenado.groupby('Fecha', as_index=False)['Resultado_cat'].max()
df_resumen.head()

,Fecha,Resultado_cat
0,2021-01-06,0
1,2021-01-11,0
2,2021-01-20,1
3,2021-01-28,2
4,2021-02-03,2


**Se completan las fechas que no tienen analisis para tener un rango temporal diario**

In [22]:
# Crear un rango de fechas diario
rango_fechas = pd.date_range(start='2021-01-01', end=df_resumen['Fecha'].max(), freq='D')

# Reindexar usando ese rango
df_completo = df_resumen.set_index('Fecha').reindex(rango_fechas).reset_index()

# Renombrar la columna de fecha
df_completo = df_completo.rename(columns={'index': 'Fecha'})

In [23]:
df_completo.head(20)

,Fecha,Resultado_cat
0,2021-01-01,NaN
1,2021-01-02,NaN
2,2021-01-03,NaN
3,2021-01-04,NaN
4,2021-01-05,NaN
5,2021-01-06,0.0
6,2021-01-07,NaN
7,2021-01-08,NaN
8,2021-01-09,NaN
9,2021-01-10,NaN


**Visualización de registros por cada categoria**

In [24]:
df_completo["Resultado_cat"].value_counts()

Resultado_cat
1.0    116
2.0     81
0.0     35
Name: count, dtype: int64

Se rellenan los vacios (donde no hay analisis) tomando el valor del analisis inmediato siguiente

In [25]:
# Rellenar 'Resultado_cat' usando forward-fill (ffill)
df_completo["Resultado_cat"] = df_completo["Resultado_cat"].ffill()

# Rellenar los nulos restantes (al inicio) con 0
df_completo["Resultado_cat"] = df_completo["Resultado_cat"].fillna(0)

print("Columna 'Resultado_cat' rellenada.")
df_completo.head()

Columna 'Resultado_cat' rellenada.


,Fecha,Resultado_cat
0,2021-01-01,0.0
1,2021-01-02,0.0
2,2021-01-03,0.0
3,2021-01-04,0.0
4,2021-01-05,0.0


**Visualizacion registros por categoria luego de rellenar los nulos**

In [26]:
df_completo["Resultado_cat"].value_counts()

Resultado_cat
1.0    689
2.0    403
0.0    368
Name: count, dtype: int64

In [27]:
df_completo['Resultado_cat'] = df_completo['Resultado_cat'].astype('int')
df_completo['Resultado_cat'] = df_completo['Resultado_cat'].astype('category')

In [28]:
df_completo.head()

,Fecha,Resultado_cat
0,2021-01-01,0
1,2021-01-02,0
2,2021-01-03,0
3,2021-01-04,0
4,2021-01-05,0


**Exportación del set de datos final**

In [29]:
df_completo.to_excel("completo_TPM.xlsx", index=False)